In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
#     dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
#                                                                     labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    #     ##################################################### REPEATING
    #     for _ in range(10):
    #         next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
    #                                        reuse=True)
    #         next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
    #                                     reuse=True)
    #         gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
    #                              action_size=action_size, reuse=True)
    #         dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    #         gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    #         aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                          labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2, (1-dones)

In [8]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones__ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates >= np.max(rates)]
            actions = actions[rates >= np.max(rates)]
            next_states = next_states[rates >= np.max(rates)]
            rewards = rewards[rates >= np.max(rates)]
            dones = dones[rates >= np.max(rates)]
            rates = rates[rates >= np.max(rates)]
            aloss, dloss, _, _ = sess.run([model.a_loss, model.d_loss, model.a_opt, model.d_opt],
                                          feed_dict = {model.states: states, 
                                                       model.actions: actions,
                                                       model.next_states: next_states,
                                                       model.rewards: rewards,
                                                       model.dones: dones,
                                                       model.rates: rates})
            aloss2, _, dones__ = sess.run([model.a_loss2, model.a_opt2, model.dones__], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            #print(dones__)
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:19.0000 R:19.0000 rate:0.0380 aloss:1.3952 dloss:0.6362 aloss2:1.3079 exploreP:0.9981
Episode:1 meanR:19.0000 R:19.0000 rate:0.0380 aloss:1.3963 dloss:0.6457 aloss2:1.2533 exploreP:0.9962
Episode:2 meanR:23.6667 R:33.0000 rate:0.0660 aloss:1.4032 dloss:0.6290 aloss2:1.2966 exploreP:0.9930
Episode:3 meanR:21.2500 R:14.0000 rate:0.0280 aloss:1.3926 dloss:0.6123 aloss2:1.2960 exploreP:0.9916
Episode:4 meanR:24.2000 R:36.0000 rate:0.0720 aloss:1.3898 dloss:0.6058 aloss2:1.3273 exploreP:0.9881
Episode:5 meanR:23.5000 R:20.0000 rate:0.0400 aloss:1.3888 dloss:0.5966 aloss2:1.3549 exploreP:0.9861
Episode:6 meanR:24.0000 R:27.0000 rate:0.0540 aloss:1.3836 dloss:0.5887 aloss2:1.3624 exploreP:0.9835
Episode:7 meanR:23.5000 R:20.0000 rate:0.0400 aloss:1.3942 dloss:0.5902 aloss2:1.3773 exploreP:0.9816
Episode:8 meanR:22.7778 R:17.0000 rate:0.0340 aloss:1.3900 dloss:0.5617 aloss2:1.4038 exploreP:0.9799
Episode:9 meanR:24.6000 R:41.0000 rate:0.0820 aloss:1.3906 dloss:0.5721 aloss2:1.3

Episode:80 meanR:27.4444 R:20.0000 rate:0.0400 aloss:1.4074 dloss:0.3752 aloss2:0.8224 exploreP:0.8027
Episode:81 meanR:27.8902 R:64.0000 rate:0.1280 aloss:1.4183 dloss:0.3866 aloss2:0.7918 exploreP:0.7976
Episode:82 meanR:27.6867 R:11.0000 rate:0.0220 aloss:1.4109 dloss:0.4148 aloss2:0.7945 exploreP:0.7967
Episode:83 meanR:27.8095 R:38.0000 rate:0.0760 aloss:1.4181 dloss:0.3955 aloss2:0.7796 exploreP:0.7938
Episode:84 meanR:28.2235 R:63.0000 rate:0.1260 aloss:1.4183 dloss:0.3810 aloss2:0.7720 exploreP:0.7888
Episode:85 meanR:28.1512 R:22.0000 rate:0.0440 aloss:1.4245 dloss:0.4039 aloss2:0.7229 exploreP:0.7871
Episode:86 meanR:28.1954 R:32.0000 rate:0.0640 aloss:1.4317 dloss:0.3946 aloss2:0.7349 exploreP:0.7846
Episode:87 meanR:30.0682 R:193.0000 rate:0.3860 aloss:1.4253 dloss:0.3808 aloss2:0.7189 exploreP:0.7698
Episode:88 meanR:29.8539 R:11.0000 rate:0.0220 aloss:1.4295 dloss:0.3874 aloss2:0.7043 exploreP:0.7690
Episode:89 meanR:29.7778 R:23.0000 rate:0.0460 aloss:1.4242 dloss:0.3734

Episode:159 meanR:47.4200 R:179.0000 rate:0.3580 aloss:1.5201 dloss:0.3911 aloss2:0.3774 exploreP:0.5405
Episode:160 meanR:48.7600 R:156.0000 rate:0.3120 aloss:1.5294 dloss:0.4041 aloss2:0.3653 exploreP:0.5323
Episode:161 meanR:49.6900 R:116.0000 rate:0.2320 aloss:1.5329 dloss:0.3995 aloss2:0.3590 exploreP:0.5263
Episode:162 meanR:49.8600 R:71.0000 rate:0.1420 aloss:1.5300 dloss:0.3941 aloss2:0.3620 exploreP:0.5226
Episode:163 meanR:50.6400 R:114.0000 rate:0.2280 aloss:1.5342 dloss:0.3999 aloss2:0.3507 exploreP:0.5168
Episode:164 meanR:53.1700 R:281.0000 rate:0.5620 aloss:1.5483 dloss:0.3847 aloss2:0.3470 exploreP:0.5028
Episode:165 meanR:52.8400 R:74.0000 rate:0.1480 aloss:1.5540 dloss:0.4158 aloss2:0.3386 exploreP:0.4991
Episode:166 meanR:52.8600 R:34.0000 rate:0.0680 aloss:1.5632 dloss:0.4072 aloss2:0.3307 exploreP:0.4975
Episode:167 meanR:53.1700 R:57.0000 rate:0.1140 aloss:1.5483 dloss:0.3863 aloss2:0.3399 exploreP:0.4947
Episode:168 meanR:54.3100 R:147.0000 rate:0.2940 aloss:1.55

Episode:237 meanR:159.9400 R:165.0000 rate:0.3300 aloss:2.1121 dloss:0.5195 aloss2:0.1333 exploreP:0.1341
Episode:238 meanR:161.5300 R:194.0000 rate:0.3880 aloss:2.0800 dloss:0.4955 aloss2:0.1376 exploreP:0.1317
Episode:239 meanR:163.6300 R:268.0000 rate:0.5360 aloss:2.0991 dloss:0.4801 aloss2:0.1279 exploreP:0.1285
Episode:240 meanR:164.2300 R:130.0000 rate:0.2600 aloss:2.1057 dloss:0.5033 aloss2:0.1203 exploreP:0.1270
Episode:241 meanR:164.8100 R:151.0000 rate:0.3020 aloss:2.1162 dloss:0.4971 aloss2:0.1261 exploreP:0.1252
Episode:242 meanR:165.7500 R:158.0000 rate:0.3160 aloss:2.1139 dloss:0.5213 aloss2:0.1295 exploreP:0.1234
Episode:243 meanR:166.6600 R:121.0000 rate:0.2420 aloss:2.0993 dloss:0.5437 aloss2:0.1320 exploreP:0.1220
Episode:244 meanR:167.6100 R:108.0000 rate:0.2160 aloss:2.1205 dloss:0.4660 aloss2:0.1321 exploreP:0.1208
Episode:245 meanR:168.6800 R:127.0000 rate:0.2540 aloss:2.1337 dloss:0.4857 aloss2:0.1273 exploreP:0.1194
Episode:246 meanR:170.1600 R:156.0000 rate:0.3

Episode:315 meanR:114.7000 R:86.0000 rate:0.1720 aloss:2.4888 dloss:0.5151 aloss2:0.1361 exploreP:0.0736
Episode:316 meanR:113.3400 R:94.0000 rate:0.1880 aloss:2.3884 dloss:0.5269 aloss2:0.1367 exploreP:0.0730
Episode:317 meanR:112.5200 R:91.0000 rate:0.1820 aloss:2.3753 dloss:0.5247 aloss2:0.1371 exploreP:0.0725
Episode:318 meanR:111.4300 R:87.0000 rate:0.1740 aloss:2.4374 dloss:0.5221 aloss2:0.1412 exploreP:0.0719
Episode:319 meanR:110.3100 R:50.0000 rate:0.1000 aloss:2.3815 dloss:0.5363 aloss2:0.1454 exploreP:0.0716
Episode:320 meanR:109.6600 R:87.0000 rate:0.1740 aloss:2.3879 dloss:0.5066 aloss2:0.1380 exploreP:0.0711
Episode:321 meanR:108.0700 R:75.0000 rate:0.1500 aloss:2.3459 dloss:0.5395 aloss2:0.1396 exploreP:0.0706
Episode:322 meanR:106.0700 R:10.0000 rate:0.0200 aloss:2.4686 dloss:0.4653 aloss2:0.1418 exploreP:0.0706
Episode:323 meanR:104.2600 R:9.0000 rate:0.0180 aloss:2.4321 dloss:0.4500 aloss2:0.1460 exploreP:0.0705
Episode:324 meanR:102.6700 R:10.0000 rate:0.0200 aloss:2

...

Episode:1274 meanR:9.6100 R:10.0000 rate:0.0200 aloss:2.9539 dloss:0.4907 aloss2:0.1023 exploreP:0.0330
Episode:1275 meanR:9.5900 R:8.0000 rate:0.0160 aloss:2.8536 dloss:0.5039 aloss2:0.1057 exploreP:0.0330
Episode:1276 meanR:9.5900 R:10.0000 rate:0.0200 aloss:2.7218 dloss:0.6423 aloss2:0.1014 exploreP:0.0330
Episode:1277 meanR:9.6200 R:11.0000 rate:0.0220 aloss:2.7763 dloss:0.4916 aloss2:0.1120 exploreP:0.0329
Episode:1278 meanR:9.6200 R:10.0000 rate:0.0200 aloss:2.8042 dloss:0.5691 aloss2:0.1262 exploreP:0.0329
Episode:1279 meanR:9.6300 R:11.0000 rate:0.0220 aloss:2.5970 dloss:0.5585 aloss2:0.1116 exploreP:0.0329
Episode:1280 meanR:9.6300 R:10.0000 rate:0.0200 aloss:2.8851 dloss:0.4916 aloss2:0.1729 exploreP:0.0329
Episode:1281 meanR:9.6400 R:10.0000 rate:0.0200 aloss:2.9024 dloss:0.6705 aloss2:0.1239 exploreP:0.0329
Episode:1282 meanR:9.6400 R:10.0000 rate:0.0200 aloss:3.0954 dloss:0.4867 aloss2:0.1147 exploreP:0.0328
Episode:1283 meanR:9.6300 R:11.0000 rate:0.0220 aloss:3.2981 dlos

Episode:1354 meanR:9.6500 R:8.0000 rate:0.0160 aloss:2.6769 dloss:0.5216 aloss2:0.0978 exploreP:0.0313
Episode:1355 meanR:9.6400 R:9.0000 rate:0.0180 aloss:2.9693 dloss:0.3710 aloss2:0.0960 exploreP:0.0313
Episode:1356 meanR:9.6600 R:11.0000 rate:0.0220 aloss:2.9576 dloss:0.3912 aloss2:0.1078 exploreP:0.0313
Episode:1357 meanR:9.6700 R:10.0000 rate:0.0200 aloss:3.1603 dloss:0.4436 aloss2:0.1062 exploreP:0.0312
Episode:1358 meanR:9.6500 R:8.0000 rate:0.0160 aloss:2.9201 dloss:0.3805 aloss2:0.1116 exploreP:0.0312
Episode:1359 meanR:9.6700 R:10.0000 rate:0.0200 aloss:2.6484 dloss:0.5610 aloss2:0.1050 exploreP:0.0312
Episode:1360 meanR:9.6800 R:9.0000 rate:0.0180 aloss:2.8563 dloss:0.4085 aloss2:0.0970 exploreP:0.0312
Episode:1361 meanR:9.6500 R:8.0000 rate:0.0160 aloss:3.0729 dloss:0.4299 aloss2:0.0861 exploreP:0.0312
Episode:1362 meanR:9.6500 R:9.0000 rate:0.0180 aloss:2.7034 dloss:0.4674 aloss2:0.1014 exploreP:0.0312
Episode:1363 meanR:9.6500 R:10.0000 rate:0.0200 aloss:2.8692 dloss:0.5

Episode:1434 meanR:9.3600 R:10.0000 rate:0.0200 aloss:2.6804 dloss:0.6377 aloss2:0.1075 exploreP:0.0298
Episode:1435 meanR:9.3700 R:11.0000 rate:0.0220 aloss:2.5653 dloss:0.6634 aloss2:0.1153 exploreP:0.0298
Episode:1436 meanR:9.3800 R:9.0000 rate:0.0180 aloss:2.9761 dloss:0.5186 aloss2:0.1026 exploreP:0.0297
Episode:1437 meanR:9.3700 R:8.0000 rate:0.0160 aloss:2.7783 dloss:0.5569 aloss2:0.1076 exploreP:0.0297
Episode:1438 meanR:9.3600 R:9.0000 rate:0.0180 aloss:2.8476 dloss:0.4983 aloss2:0.1023 exploreP:0.0297
Episode:1439 meanR:9.3500 R:9.0000 rate:0.0180 aloss:3.2396 dloss:0.4534 aloss2:0.1036 exploreP:0.0297
Episode:1440 meanR:9.3700 R:10.0000 rate:0.0200 aloss:2.8760 dloss:0.4598 aloss2:0.0968 exploreP:0.0297
Episode:1441 meanR:9.3800 R:9.0000 rate:0.0180 aloss:2.8575 dloss:0.4225 aloss2:0.0986 exploreP:0.0296
Episode:1442 meanR:9.3500 R:9.0000 rate:0.0180 aloss:2.9952 dloss:0.5077 aloss2:0.1011 exploreP:0.0296
Episode:1443 meanR:9.3700 R:11.0000 rate:0.0220 aloss:2.6043 dloss:0.6

Episode:1514 meanR:9.5400 R:9.0000 rate:0.0180 aloss:3.1280 dloss:0.3907 aloss2:0.1094 exploreP:0.0283
Episode:1515 meanR:9.5200 R:9.0000 rate:0.0180 aloss:2.8054 dloss:0.5609 aloss2:0.0972 exploreP:0.0283
Episode:1516 meanR:9.5100 R:9.0000 rate:0.0180 aloss:2.7383 dloss:0.4226 aloss2:0.1061 exploreP:0.0283
Episode:1517 meanR:9.5300 R:10.0000 rate:0.0200 aloss:3.1305 dloss:0.4476 aloss2:0.1075 exploreP:0.0283
Episode:1518 meanR:9.5500 R:10.0000 rate:0.0200 aloss:2.8435 dloss:0.4190 aloss2:0.1133 exploreP:0.0283
Episode:1519 meanR:9.5500 R:10.0000 rate:0.0200 aloss:3.0145 dloss:0.4561 aloss2:0.0916 exploreP:0.0282
Episode:1520 meanR:9.5400 R:10.0000 rate:0.0200 aloss:2.9710 dloss:0.5174 aloss2:0.1166 exploreP:0.0282
Episode:1521 meanR:9.5500 R:10.0000 rate:0.0200 aloss:3.0563 dloss:0.4257 aloss2:0.0958 exploreP:0.0282
Episode:1522 meanR:9.5400 R:9.0000 rate:0.0180 aloss:2.7039 dloss:0.4800 aloss2:0.1110 exploreP:0.0282
Episode:1523 meanR:9.5300 R:9.0000 rate:0.0180 aloss:2.8578 dloss:0.

Episode:1594 meanR:9.5200 R:9.0000 rate:0.0180 aloss:3.0284 dloss:0.5274 aloss2:0.0887 exploreP:0.0270
Episode:1595 meanR:9.5100 R:9.0000 rate:0.0180 aloss:2.7572 dloss:0.6084 aloss2:0.1069 exploreP:0.0270
Episode:1596 meanR:9.5000 R:8.0000 rate:0.0160 aloss:2.8590 dloss:0.4358 aloss2:0.0950 exploreP:0.0270
Episode:1597 meanR:9.5000 R:9.0000 rate:0.0180 aloss:3.2438 dloss:0.5279 aloss2:0.1055 exploreP:0.0269
Episode:1598 meanR:9.4900 R:8.0000 rate:0.0160 aloss:2.8263 dloss:0.5300 aloss2:0.1010 exploreP:0.0269
Episode:1599 meanR:9.4800 R:9.0000 rate:0.0180 aloss:3.3005 dloss:0.3973 aloss2:0.1055 exploreP:0.0269
Episode:1600 meanR:9.4800 R:10.0000 rate:0.0200 aloss:2.9972 dloss:0.4974 aloss2:0.1029 exploreP:0.0269
Episode:1601 meanR:9.4600 R:8.0000 rate:0.0160 aloss:2.7560 dloss:0.5205 aloss2:0.1060 exploreP:0.0269
Episode:1602 meanR:9.4600 R:10.0000 rate:0.0200 aloss:2.7884 dloss:0.4576 aloss2:0.0967 exploreP:0.0269
Episode:1603 meanR:9.4700 R:10.0000 rate:0.0200 aloss:2.6787 dloss:0.54

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.